# EDA for Train.csv

# 1. 작업 환경 설정 및 데이터 구조 확인

In [ ]:
#!pip install hvplot

In [ ]:
import pandas as pd
import hvplot
import hvplot.pandas

In [ ]:
# data load
train = pd.read_csv('../input/train.csv')

In [ ]:
# data 정보 확인
train.info()

In [ ]:
# 결측치 개수 확인
train.isnull().sum()

## 2. first_active_month
- `first_active_month`는 '처음으로 구매가 발생한 달'을 의미한다.

### 2.1 시간에 따른 first_active_month의 분포

In [41]:
# 각 first_active_month의 개수 확인
first_active_month_count = pd.DataFrame(train['first_active_month'].value_counts()).reset_index()
first_active_month_count.rename(columns = {"first_active_month":'count', 'index':'first_active_month'}, inplace=True)

# first_active_month를 기준으로 오름차순으로 정렬 후 시각화
sorted_first_active_month_count = first_active_month_count.sort_values('first_active_month')
(sorted_first_active_month_count.hvplot.line(x='first_active_month', y='count', rot=90, width=800, title="시간에 따른 first_active_month의 분포") +
sorted_first_active_month_count.hvplot.table(width=300))

:Layout
   .Curve.I :Curve   [first_active_month]   (count)
   .Table.I :Table   [first_active_month,count]

- 전체적으로 보았을 때 지속적으로 증가하다 2017년 9월 정점을 찍은 후 급격히 감소했다.
- 특정 시점에서 갑작스럽게 증가하거나 감소하는 등 급격한 변화가 있었다.
- first_active_month의 증가는 완벽하게 일치하진 않지만, 사용자의 수가 그만큼 증가했다고도 해석할 수 있다. 따라서 first_active_month의 count의 증가는 Elo의 사용자 수가 그만큼 증가했다고도 해석할 수 있다. 또한, 2018년 1월, 2월에 급격하게 하락한 이유 또한 Elo의 사용자 수와 연관 있을 것이라 추측할 수 있다.

In [42]:
# Q. 직전달과 비교했을 때 변화율이 가장 높은 구간은?
sorted_first_active_month_count['pct_change'] = sorted_first_active_month_count['count'].pct_change()
sorted_first_active_month_count.sort_values('pct_change').head()

,first_active_month,count,pct_change
62,2018-01,34,-0.996657
74,2018-02,1,-0.970588
73,2011-12,2,-0.750000
68,2012-12,18,-0.625000
67,2012-06,19,-0.321429


- 직전달과 비교했을 때 2018년 1월과 2월의 변화율이 각각 -99%, -98%로 가장 컸다.
- 이후의 EDA 과정에서 왜 이런 결과가 발생했는지 확인해 볼 필요성이 있어 보인다.

### 2.2 시간에 따른 충성도의 변화

In [39]:
grouped_first_active_month = train.groupby('first_active_month').agg({'target':'mean'}).sort_values('first_active_month').reset_index()
(grouped_first_active_month.hvplot.line(x='first_active_month', y='target', rot=90, title="시간에 따른 충성도의 변화") +
grouped_first_active_month.hvplot.table(width=300))

:Layout
   .Curve.I :Curve   [first_active_month]   (target)
   .Table.I :Table   [first_active_month,target]

In [45]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749
